In [38]:
# install libraries

%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [40]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Define connection parameters for the PostgreSQL server
db_params = {
    'user': 'postgres',
    'password': 'admin1234',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)


# Set the isolation level to AUTOCOMMIT. This change ensures that the following statements are not enclosed within a transaction.
# By setting the isolation level to ISOLATION_LEVEL_AUTOCOMMIT, 
# we effectively instructed psycopg2 to execute the database creation as a standalone operation, 
# avoiding the "CREATE DATABASE cannot run inside a transaction block" error.

conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create a new cursor
cursor = conn.cursor()

# Execute a database creation query outside of a transaction block
cursor.execute("CREATE DATABASE mydb")

# Close the cursor and the connection
cursor.close()
conn.close()


DuplicateDatabase: database "mydb" already exists


In [60]:
import psycopg2


movies_path = r'D:\DFiles\Files\Jupyter_Directory\movies_cleaned.csv'
genre_path = r'D:\DFiles\Files\Jupyter_Directory\genres_cleaned.csv'
production_companies_path = r'D:\DFiles\Files\Jupyter_Directory\production_companies_cleaned.csv'


# Define connection parameters for the PostgreSQL server
db_params = {
    'database': 'mydb',
    'user': 'postgres',
    'password': 'admin1234',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    with conn.cursor() as cursor:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS movies (
                id INTEGER PRIMARY KEY,
                adult BOOLEAN,
                budget NUMERIC,
                revenue NUMERIC,
                original_language TEXT,
                title TEXT,
                original_title TEXT,
                overview TEXT,
                popularity NUMERIC,
                release_date DATE,
                runtime NUMERIC,
                status TEXT,
                tagline TEXT,
                vote_average NUMERIC,
                vote_count INTEGER
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS genre (
                id INTEGER,
                movie_id INTEGER,
                name TEXT,
                FOREIGN KEY (movie_id) REFERENCES movies (id),
                UNIQUE (movie_id, name)
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS production_companies (
                id INTEGER,
                movie_id INTEGER,
                name TEXT,
                FOREIGN KEY (movie_id) REFERENCES movies (id),
                UNIQUE (movie_id, name)
            )
        ''')
        
        cursor.execute(f'''
            COPY movies FROM '{movies_path}' DELIMITER ',' CSV HEADER
        ''')
        
        cursor.execute(f'''
            COPY genre FROM '{genre_path}' DELIMITER ',' CSV HEADER
        ''')
        
        cursor.execute(f'''
            COPY production_companies FROM '{production_companies_path}' DELIMITER ',' CSV HEADER
        ''')
        
# Close the cursor and the connection
cursor.close()
conn.close()

# Extract all the movies that were released between January 2008 to October 2009, that have a revenue greater than their budget not including movies where the budget is listed as 0. The output should contain the following columns: id , title , release_date , budget , revenue

In [67]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        SELECT id, title, release_date, budget, revenue
        FROM movies
        WHERE release_date BETWEEN '2008-01-01' AND '2009-10-31'
          AND budget > 0
          AND revenue > budget;
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,id,title,release_date,budget,revenue
0,6557,27 Dresses,2008-01-10,30000000,160259319
1,7191,Cloverfield,2008-01-15,25000000,170764026
2,7555,Rambo,2008-01-24,50000000,113244290
3,7278,Meet the Spartans,2008-01-24,30000000,84646831
4,9030,The Eye,2008-02-01,12000000,56309766
...,...,...,...,...,...
254,15467,Kismat Konnection,2008-07-18,4180000,11000000
255,50530,The New Year's Rate Plan,2008-12-04,3200000,3877492
256,62732,Lovey-Dovey 2,2008-12-23,3000000,17850711
257,41978,High Security Vacation,2009-08-25,5000000,17566040


# Extract all the movies whose original language was not english, spanish or chinese that have a popularity rating of over 20. The output should contain the following columns: title , original_title , original_language , popularity


In [68]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        SELECT title, original_title, original_language, popularity
        FROM movies
        WHERE original_language NOT IN ('en', 'es', 'zh')
          AND popularity > 20;
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,title,original_title,original_language,popularity
0,Leon: The Professional,Léon,fr,20.477329
1,Life Is Beautiful,La vita è bella,it,39.39497
2,Spirited Away,千と千尋の神隠し,ja,41.048867
3,A Dog's Will,O Auto da Compadecida,pt,23.950396
4,Dilwale Dulhania Le Jayenge,Dilwale Dulhania Le Jayenge,hi,34.457024
5,The Mother of Tears,La terza madre,it,34.145916
6,The Key,La chiave,it,21.079551
7,The Last King,Birkebeinerne,no,50.949523
8,Train to Busan,부산행,ko,22.685169
9,Your Name.,君の名は。,ja,34.461252


# How many movies contain the word 'the' in its title? The output should contain the column movies w/ 'the' in the title

In [71]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        SELECT title
        FROM movies
        WHERE title ILIKE '%the%';
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,title
0,Father of the Bride Part II
1,The American President
2,Othello
3,Now and Then
4,The City of Lost Children
...,...
13998,The One-Man Band
13999,The Fat and Lean Wrestling Match
14000,The Morning After
14001,Shadow of the Blair Witch


# In terms of average voting rating what are the top 3 genres and top 3 production companies? The output should contain the following columns: genre, genre_average_rating, production_companies, production_company_average_rating

In [73]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        SELECT g.name AS genre, AVG(m.vote_average) AS genre_average_rating,
               pc.name AS production_company, AVG(m.vote_average) AS production_company_average_rating
        FROM genre g
        JOIN movies m ON g.movie_id = m.id
        JOIN production_companies pc ON pc.movie_id = m.id
        GROUP BY g.name, pc.name
        ORDER BY genre_average_rating DESC, production_company_average_rating DESC
        LIMIT 3;
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,genre,genre_average_rating,production_company,production_company_average_rating
0,Documentary,10.0000000000000000,Wood-Thomas Pictures,10.0000000000000000
1,Drama,10.0000000000000000,Chase Productions,10.0000000000000000
2,Fantasy,10.0000000000000000,Playhouse International Pictures,10.0000000000000000


# For each month across all years what are the top 3 genres in terms of popularity? The output should contain the following columns: month, genre, total_populartiy

In [74]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        WITH monthly_genre_popularity AS (
            SELECT DATE_TRUNC('month', m.release_date) AS month,
                   g.name AS genre,
                   SUM(m.popularity) AS total_popularity
            FROM movies m
            JOIN genre g ON g.movie_id = m.id
            GROUP BY month, genre
        )
        
        SELECT EXTRACT(MONTH FROM month) AS month, genre, total_popularity
        FROM (
            SELECT month, genre, total_popularity,
                   ROW_NUMBER() OVER (PARTITION BY month ORDER BY total_popularity DESC) AS genre_rank
            FROM monthly_genre_popularity
        ) ranked
        WHERE genre_rank <= 3
        ORDER BY month, genre_rank;
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,month,genre,total_popularity
0,1,Documentary,1.254398
1,1,Comedy,1.144203
2,1,Drama,4.224026
3,1,Fantasy,1.617271
4,1,Comedy,33.707309
...,...,...,...
3772,12,Comedy,45.202164
3773,12,Comedy,0.732804
3774,None,Science Fiction,8.942783
3775,None,Romance,8.519697


# For the top 5 production companies in terms of number of movies produced listout all the genres they have produced in that genre and the average popularity oftheir movies in that genre? The output should contain the following columns: production_company_name, genre, number_of_movies, average_popularity

In [75]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        WITH production_company_counts AS (
            SELECT pc.name AS production_company_name, COUNT(*) AS number_of_movies
            FROM production_companies pc
            GROUP BY pc.name
            ORDER BY number_of_movies DESC
            LIMIT 5
        ),
        genre_popularity AS (
            SELECT pc.name AS production_company_name, g.name AS genre, AVG(m.popularity) AS average_popularity
            FROM production_companies pc
            JOIN movies m ON pc.movie_id = m.id
            JOIN genre g ON g.movie_id = m.id
            GROUP BY pc.name, g.name
        )
        SELECT pcc.production_company_name, gp.genre, pcc.number_of_movies, gp.average_popularity
        FROM production_company_counts pcc
        LEFT JOIN genre_popularity gp ON pcc.production_company_name = gp.production_company_name
        ORDER BY pcc.number_of_movies DESC, gp.average_popularity DESC;

    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,production_company_name,genre,number_of_movies,average_popularity
0,Warner Bros.,Fantasy,1250,15.1296469902912621
1,Warner Bros.,Science Fiction,1250,12.0374743375000000
2,Warner Bros.,Adventure,1250,10.7965244031413613
3,Warner Bros.,Action,1250,10.5582974208633094
4,Warner Bros.,Thriller,1250,9.5334069920634921
...,...,...,...,...
91,Universal Pictures,Romance,830,6.1173384840764331
92,Universal Pictures,Music,830,5.6574354186046512
93,Universal Pictures,Mystery,830,5.6042352597402597
94,Universal Pictures,Western,830,3.1482879090909091


# During the summer season (June - August), what are the movies that are in the top 3 genres during that season (based on total popularity) that have a rating higher than the average rating of movies in that genre during that season.

In [76]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        WITH summer_movies AS (
            SELECT
                m.id,
                m.title,
                m.vote_average,
                g.name AS genre,
                m.popularity
            FROM movies m
            JOIN genre g ON m.id = g.movie_id
            WHERE EXTRACT(MONTH FROM m.release_date) BETWEEN 6 AND 8
        ),
        genre_avg_ratings AS (
            SELECT
                g.name AS genre,
                AVG(m.vote_average) AS avg_genre_rating
            FROM summer_movies m
            JOIN genre g ON m.id = g.movie_id
            GROUP BY g.name
        ),
        top_genres AS (
            SELECT
                g.name AS genre,
                SUM(m.popularity) AS total_popularity
            FROM summer_movies m
            JOIN genre g ON m.id = g.movie_id
            GROUP BY g.name
            ORDER BY total_popularity DESC
            LIMIT 3
        )
        SELECT
            sm.title AS movie_title,
            sm.genre AS movie_genre,
            sm.vote_average AS movie_rating,
            ga.avg_genre_rating AS genre_avg_rating,
            sm.popularity AS movie_popularity
        FROM summer_movies sm
        JOIN genre_avg_ratings ga ON sm.genre = ga.genre
        JOIN top_genres tg ON sm.genre = tg.genre
        WHERE sm.vote_average > ga.avg_genre_rating
        ORDER BY sm.genre, sm.popularity DESC;
    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,movie_title,movie_genre,movie_rating,genre_avg_rating,movie_popularity
0,Baby Driver,Action,7.2,5.6894969818913481,228.032744
1,The Dark Knight,Action,8.3,5.6894969818913481,123.167259
2,Captain Underpants: The First Epic Movie,Action,6.5,5.6894969818913481,88.561239
3,Dawn of the Planet of the Apes,Action,7.3,5.6894969818913481,75.385211
4,Boyka: Undisputed IV,Action,5.8,5.6894969818913481,67.955052
...,...,...,...,...,...
4954,The Stepmother,Drama,7.0,5.8895743301642178,0.000308
4955,Sunday Too Far Away,Drama,6.5,5.8895743301642178,0.000308
4956,The Education of Sonny Carson,Drama,7.3,5.8895743301642178,0.000308
4957,Summer,Drama,8.0,5.8895743301642178,0.00022


# Between the movies of the different runtime brackets (<30 mins, 30-60 mins, 60-120 mins, 120-180 mins, and >180 mins), what are the movies that are at least 2 times the average popularity of the movies within its bracket.

In [77]:
import pandas as pd

# Connect to the PostgreSQL server
conn = psycopg2.connect(**db_params)

# Create a new cursor
cursor = conn.cursor()

with conn:
    cursor.execute(f'''
        WITH runtime_brackets AS (
            SELECT
                id,
                title,
                runtime,
                popularity
            FROM movies
            WHERE runtime < 30
        ),
        avg_popularity AS (
            SELECT
                ROUND(AVG(popularity), 2) AS avg_popularity,
                ROUND(2 * AVG(popularity), 2) AS min_popularity
            FROM runtime_brackets
        )
        SELECT
            r.title AS movie_title,
            r.runtime AS movie_runtime,
            r.popularity AS movie_popularity,
            a.avg_popularity AS avg_popularity_in_bracket,
            a.min_popularity AS min_popularity_in_bracket
        FROM runtime_brackets r
        JOIN avg_popularity a ON TRUE
        WHERE r.popularity >= a.min_popularity;

    ''')

    # Fetch the query results and create a DataFrame
    query_results = cursor.fetchall()
    
    # Define column names
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame
    df = pd.DataFrame(query_results, columns=columns)

# Display the DataFrame
df

,movie_title,movie_runtime,movie_popularity,avg_popularity_in_bracket,min_popularity_in_bracket
0,A Grand Day Out,23.0,7.617298,0.89,1.78
1,Steamboat Willie,8.0,3.394625,0.89,1.78
2,Creature Comforts,5.0,2.329233,0.89,1.78
3,More,6.0,1.85929,0.89,1.78
4,Love the Hard Way,0.0,1.955042,0.89,1.78
...,...,...,...,...,...
319,Zygote,23.0,3.214001,0.89,1.78
320,Ducoboo,0.0,3.582418,0.89,1.78
321,Minions: Orientation Day,4.0,4.662093,0.89,1.78
322,LEGO DC Super Hero Girls: Brain Drain,0.0,8.413734,0.89,1.78


# Given a string `x` which consists of only capital letters and spaces, write a function `convert(x)` that converts the following letters to the following characters: E=3, O=0, A=4, I=1, S=$ and N=& for the given input.

In [79]:
def convert(x):
    # Create a dictionary to map characters to their replacements
    char_map = {
        'E': '3',
        'O': '0',
        'A': '4',
        'I': '1',
        'S': '$',
        'N': '&'
    }

    # Initialize an empty result string
    result = ''

    # Iterate through each character in the input string
    for char in x:
        # If the character is in the char_map, replace it; otherwise, keep it as is
        if char in char_map:
            result += char_map[char]
        else:
            result += char

    return result

# Example usage
input_string = "HELLO WORLD"
converted_string = convert(input_string)
print(converted_string)  # Output: "C0NVER$10N"


H3LL0 W0RLD


# Write a function `dupe_check(x)` that takes in a list of numbers `x` and returns a list of numbers that were duplicated (appeared in `x` at least twice). You may assume that all values in the list `x` are integers.

In [81]:
def dupe_check(x):
    # Create an empty dictionary to store the count of each number
    num_count = {}

    # Initialize an empty list to store duplicated numbers
    duplicates = []

    # Iterate through the input list
    for num in x:
        # If the number is not in the dictionary, add it with a count of 1
        if num not in num_count:
            num_count[num] = 1
        else:
            # If the number is already in the dictionary, increment its count
            num_count[num] += 1

    # Iterate through the dictionary and add numbers with count >= 2 to the duplicates list
    for num, count in num_count.items():
        if count >= 2:
            duplicates.append(num)

    return duplicates

# Example usage
input_list = [ 1, 2, 2, 4, 5, 1, 9, 10, 0, 4 ]
duplicated_numbers = dupe_check(input_list)
print(duplicated_numbers)  # Output: [2, 4, 6]


[1, 2, 4]


# Write a function `convert_time(x)` that takes in some time string in military time and outputs an AM/PM time.

In [83]:
def convert_time(military_time):
    # Split the military time string into hours and minutes
    hours, minutes = military_time.split(':')

    # Convert hours to an integer
    hours = int(hours)

    # Determine whether it's AM or PM based on the hours
    if hours < 12:
        period = "AM"
        if hours == 0:
            hours = 12  # Midnight is represented as 12:00 AM
    else:
        period = "PM"
        if hours > 12:
            hours -= 12

    # Create the AM/PM time string with a single digit for hours if needed
    am_pm_time = f"{hours}:{minutes} {period}"

    return am_pm_time

# Example usage
military_time = "14:45"
am_pm_time = convert_time(military_time)
print(am_pm_time)  # Output: "2:45 PM"


2:45 PM


In [85]:
def convert_time(military_time):
    # Split the military time string into hours and minutes
    hours, minutes = military_time.split(':')

    # Convert hours to an integer
    hours = int(hours)

    # Determine whether it's AM or PM based on the hours
    if hours < 12:
        period = "AM"
        if hours == 0:
            hours = 12  # Midnight is represented as 12:00 AM
    else:
        period = "PM"
        if hours > 12:
            hours -= 12

    # Create the AM/PM time string
    am_pm_time = f"{hours:02d}:{minutes} {period}"

    return am_pm_time

# Example usage
military_time = "14:45"
am_pm_time = convert_time(military_time)
print(am_pm_time)  # Output: "03:30 PM"


02:45 PM
